# Capstone Project Writeup

## Track
Concierge Agents

## Problem & Solution Pitch
### Problem

People often waste time checking the weather and still feel unsure about what to wear - especially in a city like Tokyo, where conditions change quickly.

### Solution

I will build a Tokyo Outfit Advisor Agent that automatically fetches the latest weather forecast and gives simple, clear clothing recommendations based on real-time conditions.

## Agent Purpose

This agent provides real-time clothing advice for walks in Tokyo based on the latest weather conditions.
It uses the Gemini 2.5 Flash-Lite model to interpret user queries, search for current weather information, and generate clear, practical recommendations.

## Features Included (Required 3+, I have 4):

・ LLM-powered agent ✔️
The main reasoning and planning steps are performed by a Gemini-based agent.

・ Built-in Google Search Tool (Grounded Web Search) ✔️
The agent retrieves live weather information for Tokyo using the integrated search tool.

・ Session & Memory Support ✔️
Through InMemorySessionService, the agent remembers earlier messages within the same session, enabling natural follow-up interactions.

・ Observability Plugins (Invocation counter + logging) ✔️
A custom plugin tracks how many times the agent is invoked. Logging is enabled for visibility during execution.

## How the Agent Works

The user asks how to dress for a walk in Tokyo.
The agent performs a grounded web search to obtain the current weather forecast and then generates clear, concise clothing advice.
If the user asks follow-up questions (e.g., what else to bring), the agent uses session memory to recall its previous recommendation and answer consistently.

## Warning!
This file was designed to execute in kaggle workspace as a neccessary part of Google/Kaggle AI Agents Intensive Course.
So if you would like to execute it in jupyter lab, you need to change it a little bit.

In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


In [3]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from google.adk.plugins.base_plugin import BasePlugin

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
# You don't need this block if execute outside kaggle workspace.
from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers

# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]['base_url']

    try:
        path_parts = baseURL.split('/')
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix

print("✅ Helper functions defined.")

✅ Helper functions defined.


In [10]:
# Observability plugin
class CountInvocationPlugin(BasePlugin):
    """Simple observability plugin that logs agent, LLM, and tool calls."""

    def __init__(self) -> None:
        super().__init__(name="count_invocation")
        self.agent_count = 0
        self.tool_count = 0
        self.llm_request_count = 0

    async def before_agent_callback(self, agent, callback_context):
        """Called before each agent run."""
        self.agent_count += 1
        print(f"[plugin] Agent runs so far: {self.agent_count}")

    async def before_model_callback(self, callback_context, llm_request, **kwargs):
        """Called before each LLM request."""
        self.llm_request_count += 1
        # llm_request is a LlmRequest object; model name is in llm_request.model
        print(f"[plugin] LLM request #{self.llm_request_count} (model={llm_request.model})")

    async def after_tool_callback(self, tool, tool_context, result):
        """Called after each tool invocation."""
        self.tool_count += 1
        print(f"[plugin] Tool '{tool.name}' completed. Total tool calls: {self.tool_count}")

In [14]:
root_agent = Agent(
    name="tokyo_outfit_agent",
    model="gemini-2.5-flash-lite",
    description="Agent that checks Tokyo weather and suggests what to wear.",
    instruction=(
        "You are a helpful assistant based in Tokyo.\n"
        "Your task is to:\n"
        "1) Use the google_search tool to check the latest weather forecast for Tokyo.\n"
        "2) Based on the forecast, recommend what the user should wear (layers, jacket, "
        "umbrella, shoes, etc.).\n"
        "3) Always answer in clear English.\n"
        "4) Never hallucinate weather data; if search fails, say so explicitly."
    ),
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


In [15]:
# Runner with plugin for observability
runner = InMemoryRunner(
    agent=root_agent,
    app_name="tokyo_weather_outfit_app",
    plugins=[CountInvocationPlugin()],
)

print("✅ Runner created.")

✅ Runner created.


In [18]:
def pretty_print(event):
    """Safely extract and print only the human-readable text from an Event."""
    try:
        content = getattr(event, "content", None)
        if not content:
            print(event)
            return

        parts = getattr(content, "parts", [])
        texts = []
        for part in parts:
            text = getattr(part, "text", None)
            if text:
                texts.append(text)

        if texts:
            print("\n".join(texts))
        else:
            print("(no text content)")
    except Exception as e:
        print(f"Error while printing event: {e}")

In [20]:
import pprint

async def run_session_example() -> None:
    # Create a session so that the agent can keep context between turns
    session = await runner.session_service.create_session(
        app_name=runner.app_name,
        user_id="demo_user",
    )

    print("=== Turn 1: initial question ===")
    first_prompt = (
        "This evening I plan to walk outside in Tokyo. "
        "Please check the latest weather forecast and tell me what I should wear."
    )

    first_events = []
    async for event in runner.run_async(
        user_id=session.user_id,
        session_id=session.id,
        new_message=types.Content(
            role="user",
            parts=[types.Part(text=first_prompt)],
        ),
    ):
        first_events.append(event)

    # Last event from the agent contains the final answer
    final_event_turn1 = [e for e in first_events if e.author == root_agent.name][-1]

    # Print the whole event structure (safe for any ADK version)
    print("---- Agent response (turn 1) ----")
    # pprint.pp(final_event_turn1)
    pretty_print(final_event_turn1)

    print("\n=== Turn 2: follow-up in the same session (tests memory) ===")
    second_prompt = (
        "I liked your outfit suggestion. Can you briefly remind me what you recommended "
        "and add one more useful item I should bring with me?"
    )

    second_events = []
    async for event in runner.run_async(
        user_id=session.user_id,
        session_id=session.id,
        new_message=types.Content(
            role="user",
            parts=[types.Part(text=second_prompt)],
        ),
    ):
        second_events.append(event)

    final_event_turn2 = [e for e in second_events if e.author == root_agent.name][-1]

    print("---- Agent response (turn 2) ----")
    # pprint.pp(final_event_turn2)
    pretty_print(final_event_turn2)


await run_session_example()

=== Turn 1: initial question ===
[plugin] Agent runs so far: 6
[plugin] LLM request #6 (model=gemini-2.5-flash-lite)
---- Agent response (turn 1) ----
The weather in Tokyo this evening is expected to be partly cloudy with a temperature around 50-55°F (10-13°C). The wind will be light.

For your walk, I recommend wearing a light jacket or a warm sweater, along with comfortable shoes. You likely won't need an umbrella.

=== Turn 2: follow-up in the same session (tests memory) ===
[plugin] Agent runs so far: 7
[plugin] LLM request #7 (model=gemini-2.5-flash-lite)
---- Agent response (turn 2) ----
I recommended wearing a light jacket or a warm sweater and comfortable shoes for your walk this evening. The weather is expected to be partly cloudy with temperatures around 50-55°F (10-13°C).

Given the temperature and the potential for a slight evening chill, it would also be useful to bring a scarf. This will help keep you warm and comfortable during your walk.
